# get data body in spark

In [1]:
data = sc.wholeTextFiles('data/reut2-*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0]).map(lambda x:x.replace('\n', ' '))

print(newsArticles.count())
newsArticles.take(3)

19043


['Paxar Corp said it has acquired Thermo-Print GmbH of Lohn, West Germany, a distributor of Paxar products, for undisclosed terms.  Reuter &#3;',
 'Shr 10 cts vs 32 cts     Net 975,000 vs 3,145,000     Sales 159.1 mln vs 147.3 mln  Reuter &#3;',
 'Key Tronic corp said it has received contracts to provide seven  original equipment manufacturers with which it has not done business recently with over 300,000 computer keyboards for delivery within the next 12 months.     The company said "The new contracts represent an annual increase of approximately 25 pct in unit volume over last year."  Reuter &#3;']

# (1) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [2]:
k = 3
shingles = newsArticles.map(lambda x:[x[i:i+k] for i in range(len(x)-k+1)])
shingles.take(1)

[['Pax',
  'axa',
  'xar',
  'ar ',
  'r C',
  ' Co',
  'Cor',
  'orp',
  'rp ',
  'p s',
  ' sa',
  'sai',
  'aid',
  'id ',
  'd i',
  ' it',
  'it ',
  't h',
  ' ha',
  'has',
  'as ',
  's a',
  ' ac',
  'acq',
  'cqu',
  'qui',
  'uir',
  'ire',
  'red',
  'ed ',
  'd T',
  ' Th',
  'The',
  'her',
  'erm',
  'rmo',
  'mo-',
  'o-P',
  '-Pr',
  'Pri',
  'rin',
  'int',
  'nt ',
  't G',
  ' Gm',
  'Gmb',
  'mbH',
  'bH ',
  'H o',
  ' of',
  'of ',
  'f L',
  ' Lo',
  'Loh',
  'ohn',
  'hn,',
  'n, ',
  ', W',
  ' We',
  'Wes',
  'est',
  'st ',
  't G',
  ' Ge',
  'Ger',
  'erm',
  'rma',
  'man',
  'any',
  'ny,',
  'y, ',
  ', a',
  ' a ',
  'a d',
  ' di',
  'dis',
  'ist',
  'str',
  'tri',
  'rib',
  'ibu',
  'but',
  'uto',
  'tor',
  'or ',
  'r o',
  ' of',
  'of ',
  'f P',
  ' Pa',
  'Pax',
  'axa',
  'xar',
  'ar ',
  'r p',
  ' pr',
  'pro',
  'rod',
  'odu',
  'duc',
  'uct',
  'cts',
  'ts,',
  's, ',
  ', f',
  ' fo',
  'for',
  'or ',
  'r u',
  ' un',
  'und',
 